## Análise do genres.csv

In [1]:
import pandas as pd
import os
import json
from essentia.standard import MonoLoader

2023-02-13 21:49:27.802149: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[   INFO   ] MusicExtractorSVM: no classifier models were configured by default
2023-02-13 21:49:28.209317: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-13 21:49:28.210931: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-13 21:49:28.257989: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemorySize: 10.76GiB dev

In [2]:
base_path = "/mnt/disks/data/fma"

connect StreamExecutor with strength 1 edge matrix:
2023-02-13 21:49:29.191483: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2023-02-13 21:49:29.191496: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2023-02-13 21:49:29.192224: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1418] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 9669 MB memory) -> physical GPU (device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5)
2023-02-13 21:49:29.193205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2023-02-13 21:49:29.193665: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-02-13 21:49:29.193686: I tensorflow/core/common_runtime/gpu

In [3]:
metadata_path = os.path.join(base_path,"fma_metadata")

width: 573.69GiB/s
2023-02-13 21:49:29.214579: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-02-13 21:49:29.214598: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-02-13 21:49:29.214603: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2023-02-13 21:49:29.214606: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2023-02-13 21:49:29.214854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1418] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 9669 MB memory) -> physical GPU (device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5)
2023-02-13 21:49:29.217848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:65:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.65GHz coreCount: 68 deviceMemo

In [4]:
dataset_path =  os.path.join(base_path,"fma_large")

## Análise do raw_genres.csv

In [5]:
raw_genres = os.path.join(metadata_path,"raw_genres.csv")

In [6]:
df_raw_genres = pd.read_csv(raw_genres)

In [7]:
df_raw_genres

,genre_id,genre_color,genre_handle,genre_parent_id,genre_title
0,1,#006666,Avant-Garde,38.0,Avant-Garde
1,2,#CC3300,International,NaN,International
2,3,#000099,Blues,NaN,Blues
3,4,#990099,Jazz,NaN,Jazz
4,5,#8A8A65,Classical,NaN,Classical
...,...,...,...,...,...
159,1032,#CC3300,Turkish,102.0,Turkish
160,1060,#CC3300,tango,46.0,Tango
161,1156,#CC3300,Fado,130.0,Fado
162,1193,#D4A017,Christmas,763.0,Christmas


In [8]:
df_raw_genres.genre_id.unique()

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         25,   26,   27,   30,   31,   32,   33,   36,   37,   38,   41,
         42,   43,   45,   46,   47,   49,   53,   58,   63,   64,   65,
         66,   70,   71,   74,   76,   77,   79,   81,   83,   85,   86,
         88,   89,   90,   92,   94,   97,   98,  100,  101,  102,  103,
        107,  109,  111,  113,  117,  118,  125,  130,  137,  138,  166,
        167,  169,  170,  171,  172,  173,  174,  175,  176,  177,  178,
        179,  180,  181,  182,  183,  184,  185,  186,  187,  188,  189,
        214,  224,  232,  236,  240,  247,  250,  267,  286,  296,  297,
        311,  314,  322,  337,  359,  360,  361,  362,  374,  377,  378,
        400,  401,  404,  428,  439,  440,  441,  442,  443,  444,  456,
        465,  468,  491,  493,  495,  502,  504,  514,  524,  538,  539,
        542,  567,  580,  602,  619,  651,  659,  6

In [9]:
df_raw_genres.genre_id.unique()

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         25,   26,   27,   30,   31,   32,   33,   36,   37,   38,   41,
         42,   43,   45,   46,   47,   49,   53,   58,   63,   64,   65,
         66,   70,   71,   74,   76,   77,   79,   81,   83,   85,   86,
         88,   89,   90,   92,   94,   97,   98,  100,  101,  102,  103,
        107,  109,  111,  113,  117,  118,  125,  130,  137,  138,  166,
        167,  169,  170,  171,  172,  173,  174,  175,  176,  177,  178,
        179,  180,  181,  182,  183,  184,  185,  186,  187,  188,  189,
        214,  224,  232,  236,  240,  247,  250,  267,  286,  296,  297,
        311,  314,  322,  337,  359,  360,  361,  362,  374,  377,  378,
        400,  401,  404,  428,  439,  440,  441,  442,  443,  444,  456,
        465,  468,  491,  493,  495,  502,  504,  514,  524,  538,  539,
        542,  567,  580,  602,  619,  651,  659,  6

## Análise do genres.csv

In [10]:
genres = os.path.join(metadata_path,"genres.csv")

In [11]:
df_genres = pd.read_csv(genres)

In [12]:
df_genres

,genre_id,#tracks,parent,title,top_level
0,1,8693,38,Avant-Garde,38
1,2,5271,0,International,2
2,3,1752,0,Blues,3
3,4,4126,0,Jazz,4
4,5,4106,0,Classical,5
...,...,...,...,...,...
158,1032,60,102,Turkish,2
159,1060,30,46,Tango,2
160,1156,26,130,Fado,2
161,1193,72,763,Christmas,38


## Análise do raw_tracks.csv

In [13]:
raw_tracks = os.path.join(metadata_path,"raw_tracks.csv")

In [14]:
df_raw_tracks = pd.read_csv(raw_tracks)

In [15]:
df_raw_tracks

,track_id,album_id,album_title,album_url,artist_id,artist_name,artist_url,artist_website,license_image_file,license_image_file_large,...,track_information,track_instrumental,track_interest,track_language_code,track_listens,track_lyricist,track_number,track_publisher,track_title,track_url
0,2,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,4656,en,1293,NaN,3,NaN,Food,http://freemusicarchive.org/music/AWOL/AWOL_-_...
1,3,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1470,en,514,NaN,4,NaN,Electric Ave,http://freemusicarchive.org/music/AWOL/AWOL_-_...
2,5,1.0,AWOL - A Way Of Life,http://freemusicarchive.org/music/AWOL/AWOL_-_...,1,AWOL,http://freemusicarchive.org/music/AWOL/,http://www.AzillionRecords.blogspot.com,http://i.creativecommons.org/l/by-nc-sa/3.0/us...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,1933,en,1151,NaN,6,NaN,This World,http://freemusicarchive.org/music/AWOL/AWOL_-_...
3,10,6.0,Constant Hitmaker,http://freemusicarchive.org/music/Kurt_Vile/Co...,6,Kurt Vile,http://freemusicarchive.org/music/Kurt_Vile/,http://kurtvile.com,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,54881,en,50135,NaN,1,NaN,Freeway,http://freemusicarchive.org/music/Kurt_Vile/Co...
4,20,4.0,Niris,http://freemusicarchive.org/music/Chris_and_Ni...,4,Nicky Cook,http://freemusicarchive.org/music/Chris_and_Ni...,NaN,http://i.creativecommons.org/l/by-nc-nd/3.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,978,en,361,NaN,3,NaN,Spiritual Level,http://freemusicarchive.org/music/Chris_and_Ni...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109722,155316,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,122,NaN,102,NaN,3,NaN,The Auger,http://freemusicarchive.org/music/Spowder/Live...
109723,155317,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,194,NaN,165,NaN,4,NaN,Let's Skin Ruby,http://freemusicarchive.org/music/Spowder/Live...
109724,155318,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,214,NaN,168,NaN,6,NaN,My House Smells Like Kim Deal/Pulp,http://freemusicarchive.org/music/Spowder/Live...
109725,155319,22940.0,"Live at Monty Hall, 2/17/2017",http://freemusicarchive.org/music/Spowder/Live...,24357,Spowder,http://freemusicarchive.org/music/Spowder/,https://spowder.bandcamp.com/,http://i.creativecommons.org/l/by-nc-nd/4.0/88...,http://fma-files.s3.amazonaws.com/resources/im...,...,NaN,0,336,NaN,294,NaN,5,NaN,The Man With Two Mouths,http://freemusicarchive.org/music/Spowder/Live...


## Análise do tracks.csv

In [16]:
tracks = os.path.join(metadata_path,"tracks.csv")

In [17]:
df_tracks = pd.read_csv(tracks,header=[1])

/tmp/ipykernel_2437642/1105780367.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tracks = pd.read_csv(tracks,header=[1])


In [18]:
df_tracks.rename(columns={"Unnamed: 0":"track_id"},inplace=True)
df_tracks.drop(index=[0],inplace=True)

In [19]:
df_tracks.reset_index(inplace=True)
df_tracks.drop(columns=['index'],inplace=True)

In [20]:
def get_structure(df,values):
    if values[2] == 0:
        return f'{values[0]}'
    else:
        return f'{values[0]}-{retorna_estrutura(df,df[df["genre_id"]==values[2]].values[0])}'

In [21]:
import re

In [22]:
df_tracks.genres.dropna(inplace=True)

In [23]:
def convert_to_int(lista):
    lista.reverse()
    value = sum([(10**i)*number for i,number in enumerate(lista)])
    return value

In [24]:
def extract_id_from_string(sentence):
    s = []
    for t in re.sub("[[]]","", sentence.split()[0]):
        try:
            s.append(int(t))
        except ValueError:
            pass
    return convert_to_int(s)

In [25]:
df_tracks["first_genre_id"] = df_tracks.genres.apply(lambda x : extract_id_from_string(x))

/tmp/ipykernel_2437642/453805256.py:3: FutureWarning: Possible nested set at position 1
  for t in re.sub("[[]]","", sentence.split()[0]):


In [26]:
df_tracks.first_genre_id.unique()

array([  21,   10,   76,   45,    1,   17,    4,   25,   26,   30,   27,
         15,   12,   31,   32,   89,   36,    3,   37,    0,   41,   43,
         79,   46,   11,   38,    2,   47,   33,   49,  250,  138,   53,
         58,   22,  103,   64,   66,   42,   70,   71,    5,   98,   74,
         85,  166,  188,   63,    9,  118,  111,  109,   97,  117,   92,
         77,   13,  137,  102,    8,  185,  101,  224,   94,   20,  125,
        180,  113,    7,  100,  906,   14,  538,    6,  107,   18,  286,
        236,   90,   88,  297,   65,  337,   86,  763,  314,  167,  172,
        179,  214,   19,  184,  181,  359,  361,  362,  183,  374,   81,
        322,  240,  186,  187,  440,  247,  171,  311,  296,  468,  182,
        495,  130, 1060,  443,   16,  542,  491,  456,  514,  169,  404,
        177, 1193,  741,  524,  360,  808,  659,  400,  189,  811,  539,
        439, 1235,  810,  442,  651,  267,   83,  504,  695,  378,  428,
        602,  444, 1032,  502,  173,  170])

## Join com tabela de generos

### Geração da hierarquia a partir das tracks


In [27]:

## Get complete genre structure
def get_all_structure(estrutura):
    ## Get structure from df_genres
    genres = os.path.join(metadata_path,"genres.csv")
    df_genres = pd.read_csv(genres)
    
    
    def get_all_structure_from_df(df_genres,estrutura):

        if estrutura == 0:
            return f'{estrutura}'
        else:
            return f'{estrutura}-{get_all_structure_from_df(df_genres,df_genres[df_genres["genre_id"]==estrutura].parent.values[0])}'
        
    
    return get_all_structure_from_df(df_genres,estrutura)
    

In [28]:
print(get_all_structure(df_tracks.iloc[158].first_genre_id))

89-25-12-0


In [29]:
df_tracks.iloc[158]

track_id                                                           346
comments                                                           0.0
date_created                                       2008-11-26 01:58:50
date_released                                      2009-01-01 00:00:00
engineer                                                           NaN
favorites                                                          1.0
id                                                               112.0
information          <p>What's this? One lion, two moles, two zoo k...
listens                                                        11086.0
producer                                                           NaN
tags                                                                []
title                                  30th Anniversary Blah Blah Blah
tracks                                                            21.0
type                                                             Album
active

In [ ]:
df_tracks["full_genre_id"] = [get_all_structure(item.first_genre_id) for i,item in df_tracks.iterrows()]

In [ ]:
df_tracks

In [ ]:
dataset_path

In [ ]:
def find_path(track_id,dataset_path):
    track_id = track_id.zfill(6)
    folder_id = track_id[0:3]
    file_path = os.path.join(dataset_path,folder_id,track_id+'.mp3')
    return file_path
    

In [ ]:
df_tracks['file_path'] = df_tracks.track_id.apply(lambda x: find_path(str(x),dataset_path))

In [ ]:
df_tracks.iloc[0].file_path

In [ ]:
def valid_music(file_path):
    try:
        # we start by instantiating the audio loader:
        loader = MonoLoader(filename=file_path)

        # and then we actually perform the loading:
        audio = loader()
        
        return True
    except:
        return False

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
df_tracks['valid'] = df_tracks.file_path.progress_apply(lambda x: valid_music(x))

In [ ]:
df_tracks.to_csv(os.path.join(metadata_path,"tracks_genres_id_full.csv"),index=False)

In [ ]:
import pandas as pd
import os

In [ ]:
df = pd.read_csv(os.path.join(metadata_path,"tracks_genres_id_full.csv"))

In [ ]:
df = df[df['valid'] == True]

In [ ]:
df.columns

In [ ]:
df = df[['file_path','first_genre_id']]

In [ ]:
df.to_csv(os.path.join(metadata_path,"tracks_genres_id_full_clear.csv"),index=False)

In [ ]:
df.columns